In [81]:
import pandas as pd
import numpy as np

At first let's perform some EDA and DQC

In [82]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
holidays_events_data = pd.read_csv('holidays_events.csv')
oil_data = pd.read_csv('oil.csv')
stores_data = pd.read_csv('stores.csv')
transactions_data = pd.read_csv('transactions.csv')

In [83]:
train_data.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [84]:
train_data.tail()

,id,date,store_nbr,family,sales,onpromotion
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8
3000887,3000887,2017-08-15,9,SEAFOOD,16.000,0


In [85]:
test_data.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [86]:
train_data.duplicated().sum()

0

In [87]:
train_data.isna().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [88]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [89]:
train_data.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [90]:
def merge_data(data):
    #data = pd.merge(data, transactions_data, on=['date', 'store_nbr'], how='inner')  !Some questions about this dataset appeared
    data = pd.merge(data, stores_data, on=['store_nbr'], how='inner')
    data = pd.merge(data, oil_data, on=['date'], how='inner')
    data = pd.merge(data, holidays_events_data, on=['date'], how='inner')
    data.rename(columns={'store_nbr' : 'store_number', 'type_x': 'store_type', 'cluster' : 'store_cluster',
                        'dcoilwtico' : 'oil_price', 'locale' : 'holiday_status', 'locale_name' : 'holiday_location',
                        'description' : 'holiday_description', 'type_y' : 'day_type', 'transferred' : 'is_holiday_transferred',
                        'family' : 'item_family', 'sales' : 'item_sales', 'onpromotion' : 'items_on_promotion'}, inplace=True)
    data['date'] = pd.to_datetime(data['date'])  
    data.set_index('date', inplace=True)
    data.drop(['id'], axis=1, inplace=True)
    return data


In [91]:
train_data = merge_data(train_data)
test_data = merge_data(test_data)

In [92]:
train_data.head()

,store_number,item_family,item_sales,items_on_promotion,city,state,store_type,store_cluster,oil_price,day_type,holiday_status,holiday_location,holiday_description,is_holiday_transferred
date,,,,,,,,,,,,,,
2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False


In [93]:
test_data.head()

,store_number,item_family,items_on_promotion,city,state,store_type,store_cluster,oil_price,day_type,holiday_status,holiday_location,holiday_description,is_holiday_transferred
date,,,,,,,,,,,,,
2017-08-24,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,47.24,Holiday,Local,Ambato,Fundacion de Ambato,False
2017-08-24,1,BABY CARE,0,Quito,Pichincha,D,13,47.24,Holiday,Local,Ambato,Fundacion de Ambato,False
2017-08-24,1,BEAUTY,0,Quito,Pichincha,D,13,47.24,Holiday,Local,Ambato,Fundacion de Ambato,False
2017-08-24,1,BEVERAGES,26,Quito,Pichincha,D,13,47.24,Holiday,Local,Ambato,Fundacion de Ambato,False
2017-08-24,1,BOOKS,0,Quito,Pichincha,D,13,47.24,Holiday,Local,Ambato,Fundacion de Ambato,False
